<a href="https://colab.research.google.com/github/im-Lily/Convergence_project/blob/develop/yolo_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
# yolov5 폴더가 만들어진 경로를 찾아서 들어가주세요
%cd yolov5

In [ ]:
# 데이터셋 압축해제
!unzip -q ../train_data.zip -d ../

In [ ]:
# 필요 라이브러리 설치
!pip install -qr requirements.txt

In [ ]:
# coco128.yaml 파일 내용 확인
%cat /content/yolov5/data/coco128.yaml

In [ ]:
# base_img 개수 확인
%cd /content/
from glob import glob

img_list = glob('/content/train_data/images/train/*.jpg')

print(len(img_list))

In [ ]:
# 데이터셋 분할
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

In [ ]:
# train, val dataset list 확인
with open('/content/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [ ]:
import yaml

with open('/content/yolov5/data/coco128.yaml', 'r') as f:
  data = yaml.load(f)

print(data)

data['train'] = '/content/train.txt'
data['val'] = '/content/val.txt'
data['nc'] = 3
data['names'] = ['PET','PP','PS']


with open('/content/yolov5/data/coco128.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

In [ ]:
# train
%cd /content/yolov5/

!python train.py --img 640 --batch 16 --epochs 50 --data /data/coco128.yaml --weights yolov5m.pt --cfg ./models/yolov5m.yaml

In [ ]:
from IPython.display import Image
import os

val_img_path = val_img_list[4]

!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.5 --source "{val_img_path}"

Image(os.path.join('/content/yolov5/runs/detect/exp', os.path.basename(val_img_path)))

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/

In [ ]:
!python test.py --data coco128.yaml --weights ./runs/train/exp/weights/best.pt --save-json

In [ ]:
# test dataset 압축해제
!unzip -q ../mark.zip -d ../

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source ./uploads/test1.jpg

In [ ]:
!python custom_detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source ./uploads/test.jpg